In [2]:
:dep ndarray = "0.15.6"
:dep ndarray-rand = "0.14.0"
:dep showata = { version = "0.3.2", features=["show_ndarray"]}
:dep polars = { version = "0.29.0", features=["ndarray"]}

In [3]:
use ndarray::{Array, Array2, stack, concatenate};
use ndarray_rand::RandomExt;
use ndarray_rand::rand_distr::Uniform;
use ndarray::prelude::*;

In [4]:
use std::f64;

#[derive(Debug)]
enum ActivationFunction {
    Tanh,
}

impl ActivationFunction {
    fn activate(&self, x: f64) -> f64 {
        match self {
            ActivationFunction::Tanh => self.tanh(x),
        }
    }
    
    fn derivative(&self, x: f64) -> f64 {
        match self {
            ActivationFunction::Tanh => self.tanh_derivative(x),
        }
    }
    
    fn tanh(&self, x: f64) -> f64 {
        x.tanh()
    }
    
    fn tanh_derivative(&self, x: f64) -> f64 {
        1.0 - self.tanh(x).powi(2)
    }
}



In [5]:
#[derive(Debug)]
struct Layer {
    len_inputs: usize,
    neurons: usize,
    function: ActivationFunction,
    weights: Array2<f64>,
    input: Option<Array2<f64>>,
    net: Option<Array2<f64>>,
    output: Option<Array2<f64>>,
    idx: Option<usize>,
}

impl Layer {
    fn new(len_inputs: usize, neurons: usize, function: ActivationFunction) -> Self {
        let shape = (neurons, len_inputs + 1);
        let weights = Array::random(shape, Uniform::new(-0.5, 0.5));

        Layer {
            len_inputs: len_inputs,
            neurons: neurons,
            function: function,
            weights: weights,
            input: None,
            net: None,
            output: None,
            idx: None,
        }
    }

    fn forward(&mut self, layer_input: &Array2<f64>) -> Array2<f64> {
        self.input = Some(layer_input.clone());
        self.net = Some(self.input.as_ref().unwrap().dot(&self.weights.t()));
        self.output = Some(self.net.as_ref().unwrap().mapv(|x| self.function.activate(x)));
        self.output.clone().unwrap()
    }

    
    fn backward(
        &mut self, 
        alpha: f64,
        last: bool,
        previous_delta: Option<&Array2<f64>>,
        previous_weight: Option<&Array2<f64>>,
        error: Option<&Array2<f64>>
    ) -> (Array2<f64>, Array2<f64>) {
        
        let delta = if last {
            let error = error.unwrap();
            error * self.net.as_ref().unwrap().mapv(|x| self.function.derivative(x))
        } else {
            let previous_delta = previous_delta.unwrap();
            let previous_weight = previous_weight.unwrap();
            let delta = previous_delta.dot(previous_weight).slice(s![.., 1..]).to_owned();
            delta * self.net.as_ref().unwrap().mapv(|x| self.function.derivative(x))
        };
        
        
        self.weights = delta.t().dot(self.input.as_ref().unwrap()) * alpha + &self.weights;
        
        (delta, self.weights.to_owned())
    }



    fn set_idx(&mut self, idx: usize) {
        self.idx = Some(idx);
    }
}

In [6]:
#[derive(Debug)]
struct NeuralNetwork {
    layers: Vec<Layer>,
    all_mse: Vec<f64>,
}

impl NeuralNetwork {
    fn new(mut layers: Vec<Layer>) -> Self {
        for (idx, layer) in layers.iter_mut().enumerate() {
            layer.set_idx(idx + 1);
        }
        
        NeuralNetwork {
            layers,
            all_mse: Vec::new(),
        }
    }

    fn forward(&mut self, x_input: &Array2<f64>) -> Array2<f64> {
        let mut input_layer = concatenate![Axis(1), Array::from_shape_vec((1, 1), vec![1.0]).unwrap(), x_input.clone()];
        let mut output: Array2<f64> = Array::zeros((0, 0));

        for layer in &mut self.layers {
            output = layer.forward(&input_layer);
            input_layer = concatenate![Axis(1), Array::from_shape_vec((1, 1), vec![1.0]).unwrap(), output];
        }

        output
    }

    fn backward(&mut self, alpha: f64, error: &Array2<f64>) {
        let mut previous_delta = None;
        let mut previous_weight = None;
        let mut last = true;
        
        for layer in self.layers.iter_mut().rev() {
            let (delta, weights) = layer.backward(alpha, last, previous_delta.as_ref(), previous_weight.as_ref(), Some(&error));
            last = false;
            previous_delta = Some(delta);
            previous_weight = Some(weights);
        }
    }

    fn predict(&mut self, x: &Array2<f64>) -> Array2<f64> {
        self.forward(x)
    }
}

In [617]:
let weights_1 = array![[0.2, 0.4, 0.5],[0.3, 0.6, 0.7],[0.4, 0.8, 0.3]];
let weights_2 = array![[-0.7, 0.6, 0.2, 0.7],[-0.3, 0.7, 0.2, 0.8]];
let weights_3 = array![[0.1, 0.8, 0.5]];


let mut nn = NeuralNetwork::new(vec![
    Layer::new(2, 3, ActivationFunction::Tanh),
    Layer::new(3, 2, ActivationFunction::Tanh),
    Layer::new(2, 1, ActivationFunction::Tanh),
]);

nn.layers[0].weights = weights_1;
nn.layers[1].weights = weights_2;
nn.layers[2].weights = weights_3;

let x_inputs = array![[0.3, 0.7]];
let out = nn.forward(&x_inputs);
out


[[0.5763660324621392]], shape=[1, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2

In [618]:
nn.backward(0.05, &array![[-2.0]]);
nn.predict(&x_inputs)

[[0.403346832835286]], shape=[1, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2

In [9]:
use polars::prelude::*;
let file_path = "train.csv";

    // Ler o arquivo CSV para um DataFrame
let df: DataFrame = CsvReader::from_path(file_path)
    .unwrap()
    .infer_schema(None)
    .has_header(true)
    .finish()
    .unwrap();

// Exibir o DataFrame

In [110]:
:dep ndarray-csv = {version = "0.4.1"}
:dep csv

In [10]:
let ndarray = df.to_ndarray::<Float64Type>().unwrap();

In [33]:
let mut z = Array::zeros((1, ndarray.shape()[1]-1));
for linha in ndarray.slice(s![.., 1..]).to_owned().axis_iter(Axis(0)) {
    z.assign(&linha);
}
z

[[0.0, 0.0, 0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0, 0.0, 0.0]], shape=[1, 784], strides=[784, 1], layout=CFcf (0xf), const ndim=2

In [18]:
let y_train = ndarray.slice(s![.., ..1]);
let x_train = ndarray.slice(s![.., 1..]).to_owned()

Error: The variable `y_train` has type `ArrayBase<ndarray::ViewRepr<&f64>, Dim<[usize; _]>>` which cannot be persisted.
You might be able to fix this by creating a `Box<dyn YourType>`. e.g.
let v: Box<dyn core::fmt::Debug> = Box::new(foo());
Alternatively, you can prevent evcxr from attempting to persist
the variable by wrapping your code in braces.

In [ ]:
ndarray.slice(s![..10, 1..]).to_owned().iter().map(|x| x*2.0).collect::<Vec<f64>>()

In [7]:

let mut nn = NeuralNetwork::new(vec![
    Layer::new(784, 20, ActivationFunction::Tanh),
    Layer::new(20, 20, ActivationFunction::Tanh),
    Layer::new(20, 10, ActivationFunction::Tanh),
]);




In [8]:
nn.layers[2].weights

[[0.40586170804815636, -0.024131547821892196, -0.44243107583669894, -0.16584735920113047, 0.10727349840600886, 0.07698606477674463, -0.16665875216878479, 0.2790247212350334, 0.4033494545401226, -0.47144016262747224, 0.36555478020993704, -0.4810957879433184, 0.36594020420818363, -0.12174270143021482, -0.41494033382247575, -0.3384689959641929, -0.17831365779862485, -0.05846385876718263, 0.04411660836506481, -0.24785998368543627, 0.04180966107104611],
 [0.4964477653319561, -0.3470791563261766, -0.2072082184065518, -0.21657467222091142, 0.10653108531627398, 0.2603941293147294, 0.30055876061308373, -0.4863070543958341, 0.05346270687151278, -0.44176736771713654, 0.2170278690101386, -0.27908065697464934, -0.4877812359309186, -0.08193316103218584, 0.4556761955186934, -0.42403803361746295, 0.11094675856660641, -0.17523237885043952, -0.33944500490268603, 0.15043346385477685, -0.463497056476472],
 [-0.41644619945209893, 0.17696931011206707, 0.2298025343451442, 0.09830706504452102, -0.255508188108

In [34]:
nn.forward(&z)

[[-0.8575975543026888, 0.7418844757167533, 0.008276932948479847, 0.30835947317996804, -0.9020352534626893, 0.7557398196992497, 0.7035063310380174, -0.7711807842923268, 0.5960649283081518, 0.9926227474377259]], shape=[1, 10], strides=[1, 1], layout=CFcf (0xf), const ndim=2